# Genshin Calculator:

In [1]:
class Character:
     def __init__(self, name, level, vision_element, damage_attribute = None, survival_attribute = None):
        self.name = name
        self.level = level       
        self.vision_element = vision_element
        self.damage_attribute = damage_attribute                    
        self.survival_attribute = survival_attribute 

In [2]:
class Damage_attribute:
    def __init__(self, atk, elemental_mastery, crit_rate, crit_damage, cd_reduction, dmg_bonus):
        self.atk = atk
        self.elemental_mastery = elemental_mastery
        self.crit_rate = crit_rate                        #to get the tolerance for the norm of gradients 
        self.crit_damage = crit_damage                    #maximum number of iteration of gradient descent
        self.cd_reduction = cd_reduction
        self.dmg_bonus = dmg_bonus
    
    
    def show(self):                              # print method
        print(f'damage attribute: ')
        print(f'  atk:               {self.atk}')
        print(f'  elemental_mastery: {self.elemental_mastery}')
        print(f'  crit_rate:         {self.crit_rate}')
        print(f'  crit_damage:       {self.crit_damage}')
        print(f'  cd_reduction:      {self.cd_reduction}')
        print(f'  dmg_bonus:         {self.dmg_bonus}')

# Test:

In [3]:
dmg_att = Damage_attribute(1930, 20, 45, 180, 0, 90)

In [4]:
dmg_att.show()

damage attribute: 
  atk:               1930
  elemental_mastery: 20
  crit_rate:         45
  crit_damage:       180
  cd_reduction:      0
  dmg_bonus:         90


In [5]:
childe = Character('Childe', 90, 'Hydro', dmg_att)

In [6]:
childe.damage_attribute.show()

damage attribute: 
  atk:               1930
  elemental_mastery: 20
  crit_rate:         45
  crit_damage:       180
  cd_reduction:      0
  dmg_bonus:         90
